In [1]:
%%capture
# Normally using pip install unsloth is enough

# Temporarily as of Jan 31st 2025, Colab has some issues with Pytorch
# Using pip install unsloth will take 3 minutes, whilst the below takes <1 minute:
!pip install --no-deps bitsandbytes accelerate xformers==0.0.29 peft trl triton
!pip install --no-deps cut_cross_entropy unsloth_zoo
!pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
!pip install --no-deps unsloth

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [2]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 15 trillion tokens model 2x faster!
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # We also uploaded 4bit for 405b!
    "unsloth/Mistral-Nemo-Base-2407-bnb-4bit", # New Mistral 12b 2x faster!
    "unsloth/Mistral-Nemo-Instruct-2407-bnb-4bit",
    "unsloth/mistral-7b-v0.3-bnb-4bit",        # Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",           # Phi-3.5 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 02-16 14:35:06 __init__.py:190] Automatically detected platform cuda.
==((====))==  Unsloth 2025.2.12: Fast Llama patching. Transformers: 4.48.3.
   \\   /|    GPU: NVIDIA A100-PCIE-40GB. Max memory: 39.394 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1. CUDA: 8.0. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [3]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN

In [5]:
import json

import time

def load_json(file_paths):
    data = []
    for file_path in file_paths:
        with open(file_path, "r", encoding="utf-8") as file:
            data.extend(json.load(file))
    return data

def save_to_jsonl(file_path, data):
    with open(file_path, "a", encoding="utf-8") as f: 
        f.write(json.dumps(data, ensure_ascii=False) + "\n")

def generate_response(instruction_text, input_text, max_tokens):
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference
    inputs = tokenizer(
    [
        alpaca_prompt.format(
            instruction_text, # instruction
            input_text, # input
            "", # output - leave this blank for generation!
        )
    ], return_tensors = "pt").to("cuda")

    outputs = model.generate(**inputs, 
                             max_new_tokens=max_tokens, 
                             use_cache=True,)
    decoded_outputs = tokenizer.batch_decode(outputs)

    response_texts = [output.split("### Response:\n")[-1].strip() for output in decoded_outputs]

    return response_texts[0].replace("<|eot_id|>", "")

# 실행 
input_json_path = input_json_paths = [
    "/data/jaesung/llm_for_diabetes/src/model/unsloth/test_dataset/data_classification_test.json",
    "/data/jaesung/llm_for_diabetes/src/model/unsloth/test_dataset/data_generation_test.json"
]
output_json_path = "/data/jaesung/llm_for_diabetes/src/model/unsloth/model_output/zero_shot_model_output_2.jsonl"

data = load_json(input_json_path)

# temp
data = [item for item in data if item.get("task") == "summarization"]


start_time = time.time()
total_samples = len(data)
for idx, item in enumerate(data):
    sample_start_time = time.time()

    input_text = item.get("input", "")
    instruction = item.get("instruction", "")
    task = item.get("task", "")

    max_tokens = (32 if task in ["qa1", "qa2", "qa3", "nli", "ie", "re"] 
                  else 2048 if task in ["summarization"] 
                  else 512)

    model_output = generate_response(instruction, input_text, max_tokens)

    output_data = item.copy()
    output_data.update({
        "model_output": model_output,
    })
    save_to_jsonl(output_json_path, output_data)

    elapsed_time = time.time() - start_time
    avg_time_per_sample = elapsed_time / (idx + 1) 
    remaining_samples = total_samples - (idx + 1)
    estimated_remaining_time = remaining_samples * avg_time_per_sample

    print(f"[{idx+1}/{total_samples}] Sample processed in {time.time() - sample_start_time:.2f}s, ETA: {estimated_remaining_time/60:.2f} min")

print(f"\nAll samples processed. Total time: {(time.time() - start_time)/60:.2f} min")

[1/100] Sample processed in 18.14s, ETA: 29.93 min
[2/100] Sample processed in 58.15s, ETA: 62.30 min
[3/100] Sample processed in 16.80s, ETA: 50.16 min
[4/100] Sample processed in 14.11s, ETA: 42.88 min
[5/100] Sample processed in 58.53s, ETA: 52.48 min
[6/100] Sample processed in 58.90s, ETA: 58.65 min
[7/100] Sample processed in 14.93s, ETA: 53.04 min
[8/100] Sample processed in 58.88s, ETA: 57.20 min
[9/100] Sample processed in 9.67s, ETA: 51.92 min
[10/100] Sample processed in 58.81s, ETA: 55.04 min
[11/100] Sample processed in 11.16s, ETA: 50.98 min
[12/100] Sample processed in 13.55s, ETA: 47.86 min
[13/100] Sample processed in 11.11s, ETA: 44.92 min
[14/100] Sample processed in 13.55s, ETA: 42.62 min
[15/100] Sample processed in 59.03s, ETA: 44.89 min
[16/100] Sample processed in 59.32s, ETA: 46.78 min
[17/100] Sample processed in 59.04s, ETA: 48.31 min
[18/100] Sample processed in 59.04s, ETA: 49.56 min
[19/100] Sample processed in 59.19s, ETA: 50.58 min
[20/100] Sample proces

KeyboardInterrupt: 

: 

In [15]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Analyze the sentence with two drugs labeled as @DRUG_A$ and @DRUG_B$. Extract the interaction between @DRUG_A$ and @DRUG_B$ from the input sentence by selecting only one of the following options: 'DDI-effect', 'DDI-mechanism', 'DDI-advise', 'DDI-false', and 'DDI-int'. 'DDI-effect': Choose this when the interaction describes an effect or a pharmacodynamic mechanism. 'DDI-mechanism': Choose this for interactions explained by pharmacokinetic mechanisms. 'DDI-advise': Choose this when the sentence provides a recommendation or advice about the drug interaction. 'DDI-false': Choose this if there is no actual drug-drug interaction in the sentence. 'DDI-int': Choose this when a drug-drug interaction is mentioned without additional detail.", # instruction
        "@DRUG_A$ did not interfere with the absorption or disposition of the @DRUG_B$ glyburide in diabetic patients.", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens=64, use_cache=True)
decoded_outputs = tokenizer.batch_decode(outputs)

response_texts = [output.split("### Response:\n")[-1].strip() for output in decoded_outputs]

print(response_texts[0]) 


DDI-false<|eot_id|>
